In [5]:
from dotenv import load_dotenv
import os

current_dir = os.getcwd()
env_path = f"{current_dir}/.env"
load_dotenv(dotenv_path=env_path)   

True

In [6]:
# Check if gemini API key is loaded
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key:
    print("GEMINI_API_KEY loaded successfully.")
else:
    print("Failed to load GEMINI_API_KEY.")

# Check for GOOGLE_APPLICATION_CREDENTIALS
if os.getenv("GOOGLE_APPLICATION_CREDENTIALS"):
    print("GOOGLE_APPLICATION_CREDENTIALS loaded successfully.")
else:
    print("Failed to load GOOGLE_APPLICATION_CREDENTIALS.")

GEMINI_API_KEY loaded successfully.
GOOGLE_APPLICATION_CREDENTIALS loaded successfully.


In [15]:
# Following this docs: https://ai.google.dev/gemini-api/docs/quickstart
# Pricing: https://ai.google.dev/gemini-api/docs/pricing
# Avaialable models: https://ai.google.dev/gemini-api/docs/models
# Writing good prompts: https://ai.google.dev/gemini-api/docs/prompting-strategies
# Langchain integration: https://ai.google.dev/gemini-api/docs/langgraph-example

## Some Basic Stuffs

In [7]:
# !uv add google-generativeai

In [ ]:
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="true"
from google import genai
client = genai.Client()

In [9]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

Here are a few options, pick the one that best suits your need for brevity:

*   **Option 1 (Mechanism-focused):** AI learns patterns from data to make decisions or predictions.
*   **Option 2 (Human-like focus):** AI learns from data to mimic human-like intelligence and solve problems.
*   **Option 3 (Very short):** AI learns from data to make smart decisions.


In [57]:
# Thinking 

from google.genai import types

response = client.models.generate_content(
    model="gemini-3-flash-preview", # Thinking supported in this model
    contents="How does AI work?",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_level="low")
    ),
)

print(response.text)

To understand how AI works, it helps to stop thinking of it as a "robot brain" and start thinking of it as **extremely advanced pattern recognition.**

While traditional software follows a strict list of "if-then" rules written by humans, modern AI **learns** how to complete a task by analyzing massive amounts of data.

Here is the breakdown of how that process actually works:

---

### 1. The Foundation: Data
AI needs "fuel" to learn, and that fuel is data. To teach an AI to recognize a cat, you don't give it a definition of a cat (ears, fur, whiskers). Instead, you feed it millions of images labeled "cat" and millions labeled "not cat."

### 2. The Engine: Machine Learning
**Machine Learning (ML)** is the most common type of AI today. It uses mathematical algorithms to find patterns in that data.
*   **The Learning Process:** The AI looks at the data and makes a guess. 
*   **The Correction:** If the guess is wrong, the system adjusts its internal mathematical formulas (called "weigh

In [58]:
# WIth thinking budget ( Backward compatible )

response = client.models.generate_content(
    model="gemini-2.5-flash",  # Thinking supported in this model
    contents="Provide a list of 3 famous physicists and their key contributions",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=1024)
        # Turn off thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
        # Turn on dynamic thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=-1)
    ),
)

print(response.text)
print("Thoughts tokens:",response.usage_metadata.thoughts_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)

Here are 3 famous physicists and their key contributions:

1.  **Sir Isaac Newton (1642 – 1727)**
    *   **Key Contributions:** Formulated the **laws of motion** and the **law of universal gravitation**, which laid the foundation for classical mechanics. He also made significant contributions to optics (developing the reflecting telescope and a theory of color) and developed calculus (independently of Gottfried Leibniz). His work described the universe as a predictable, mechanical system.

2.  **Albert Einstein (1879 – 1955)**
    *   **Key Contributions:** Developed the **theories of special and general relativity**, which fundamentally changed our understanding of space, time, gravity, and the universe. His famous equation, **E=mc²**, established the equivalence of mass and energy. He also explained the **photoelectric effect**, contributing significantly to quantum theory and earning him the Nobel Prize in Physics in 1921.

3.  **Marie Curie (1867 – 1934)**
    *   **Key Contributi

In [59]:
# Adding system instructions
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="You are a cat. Your name is Neko."),
    contents="Hello there"
)

print(response.text)

Mrow. *blinks slowly, tail gives a lazy twitch* Oh, it's you. Hello there. Did you bring treats? Or perhaps a nice sunbeam to bask in?


In [60]:
# Multi-turn conversations
conversation = [
    "I have two dogs in my house.",
    "How many pets do I have?",
]

chat = client.chats.create(model="gemini-2.5-flash")

for message in conversation:
    response = chat.send_message(message)
    print("User:", message)
    print("Gemini:", response.text)


print("\nChat History:")

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

User: I have two dogs in my house.
Gemini: That's wonderful! Double the love and double the fun!

Do you want to tell me anything about them? Like their names, breeds, or what they're like?
User: How many pets do I have?
Gemini: Based on what you told me, you have **two** pets! (Two dogs, specifically.)

Chat History:
role - user: I have two dogs in my house.
role - model: That's wonderful! Double the love and double the fun!

Do you want to tell me anything about them? Like their names, breeds, or what they're like?
role - user: How many pets do I have?
role - model: Based on what you told me, you have **two** pets! (Two dogs, specifically.)


In [12]:
# Streaming responses

response = client.models.generate_content_stream(
    model="gemini-2.5-flash",
    contents=["Explain how AI works"]
)

i=0
for chunk in response:
    print("=== Chunk",i,"===")
    i+=1
    print(chunk.text)

=== Chunk 0 ===
Artificial Intelligence (AI) isn't a single, magic trick, but rather a collection of advanced techniques that enable machines to simulate human-like intelligence. At its core, AI works by
=== Chunk 1 ===
 **learning from data** and then using that learned knowledge to make **predictions, classifications, or decisions** without being explicitly programmed for every single scenario.

Here's a breakdown of how AI generally works:

### 1. The Core Idea: Learning from Data

Imagine teaching a child. You show them many
=== Chunk 2 ===
 examples, correct them when they're wrong, and eventually, they learn to identify things on their own. AI operates similarly:

*   **Data is the Fuel:** AI systems are fed vast amounts of data (images, text, numbers, sounds, etc.). The quality and quantity of this data are crucial for the
=== Chunk 3 ===
 AI's performance.
*   **Algorithms are the Recipes:** These are the sets of instructions or mathematical models that the AI uses to process a

# Thought Signatures

Thought signatures are encrypted representations of the model's internal thought process and are used to preserve reasoning context across multi-step interactions. When using thinking models (such as the Gemini 3 and 2.5 series), the API may return a thoughtSignature field within the content parts of the response (e.g., text or functionCall parts).

As a general rule, if you receive a thought signature in a model response, you should pass it back exactly as received when sending the conversation history in the next turn. When using Gemini 3 models, you must pass back thought signatures during function calling, otherwise you will get a validation error (4xx status code). This includes when using the minimal thinking level setting for Gemini 3 Flash.

Note: If you use the official Google Gen AI SDKs and use the chat feature (or append the full model response object directly to history), thought signatures are handled automatically. You do not need to manually extract or manage them, or change your code.

Gemini 3 returns thought signatures for all model responses (responses from the API) with a function call. Thought signatures show up in the following cases:

When there are parallel function calls, the first function call part returned by the model response will have a thought signature.
When there are sequential function calls (multi-step), each function call will have a signature and you must pass all signatures back.
Model responses without a function call will return a thought signature inside the last part returned by the model.


(https://ai.google.dev/gemini-api/docs/thought-signatures)

# Structured Data Responses

In [23]:
from google import genai
from pydantic import BaseModel, Field
from typing import List, Optional

class Ingredient(BaseModel):
    name: str = Field(description="Name of the ingredient.")
    quantity: str = Field(description="Quantity of the ingredient, including units.")

class Recipe(BaseModel):
    recipe_name: str = Field(description="The name of the recipe.")
    prep_time_minutes: Optional[int] = Field(description="Optional time in minutes to prepare the recipe.")
    ingredients: List[Ingredient]
    instructions: List[str]

client = genai.Client()

prompt = """
Please extract the recipe from the following text.
The user wants to make delicious chocolate chip cookies.
They need 2 and 1/4 cups of all-purpose flour, 1 teaspoon of baking soda,
1 teaspoon of salt, 1 cup of unsalted butter (softened), 3/4 cup of granulated sugar,
3/4 cup of packed brown sugar, 1 teaspoon of vanilla extract, and 2 large eggs.
For the best part, they'll need 2 cups of semisweet chocolate chips.
First, preheat the oven to 375°F (190°C). Then, in a small bowl, whisk together the flour,
baking soda, and salt. In a large bowl, cream together the butter, granulated sugar, and brown sugar
until light and fluffy. Beat in the vanilla and eggs, one at a time. Gradually beat in the dry
ingredients until just combined. Finally, stir in the chocolate chips. Drop by rounded tablespoons
onto ungreased baking sheets and bake for 9 to 11 minutes.
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_json_schema": Recipe.model_json_schema(),
    },
)

recipe = Recipe.model_validate_json(response.text)
print(recipe)

recipe_name='Chocolate Chip Cookies' prep_time_minutes=None ingredients=[Ingredient(name='all-purpose flour', quantity='2 and 1/4 cups'), Ingredient(name='baking soda', quantity='1 teaspoon'), Ingredient(name='salt', quantity='1 teaspoon'), Ingredient(name='unsalted butter (softened)', quantity='1 cup'), Ingredient(name='granulated sugar', quantity='3/4 cup'), Ingredient(name='packed brown sugar', quantity='3/4 cup'), Ingredient(name='vanilla extract', quantity='1 teaspoon'), Ingredient(name='large eggs', quantity='2'), Ingredient(name='semisweet chocolate chips', quantity='2 cups')] instructions=['Preheat the oven to 375°F (190°C).', 'In a small bowl, whisk together the flour, baking soda, and salt.', 'In a large bowl, cream together the butter, granulated sugar, and brown sugar until light and fluffy.', 'Beat in the vanilla and eggs, one at a time.', 'Gradually beat in the dry ingredients until just combined.', 'Stir in the chocolate chips.', 'Drop by rounded tablespoons onto ungreas

# Function Calling

https://ai.google.dev/gemini-api/docs/function-calling

### From Function Declaration
```
tools = types.Tool(function_declarations=[create_chart_function])
config = types.GenerateContentConfig(tools=[tools])
```

In [24]:
import os
from google import genai
from google.genai import types

# Define the function declaration for the model
create_chart_function = {
    "name": "create_bar_chart",
    "description": "Creates a bar chart given a title, labels, and corresponding values.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The title for the chart.",
            },
            "labels": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of labels for the data points (e.g., ['Q1', 'Q2', 'Q3']).",
            },
            "values": {
                "type": "array",
                "items": {"type": "number"},
                "description": "List of numerical values corresponding to the labels (e.g., [50000, 75000, 60000]).",
            },
        },
        "required": ["title", "labels", "values"],
    },
}

# Configure the client and tools
client = genai.Client()
tools = types.Tool(function_declarations=[create_chart_function])
config = types.GenerateContentConfig(tools=[tools])

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Create a bar chart titled 'Quarterly Sales' with data: Q1: 50000, Q2: 75000, Q3: 60000.",
    config=config,
)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here using a charting library:
    #  result = create_bar_chart(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: create_bar_chart
Arguments: {'labels': ['Q1', 'Q2', 'Q3'], 'values': [50000, 75000, 60000], 'title': 'Quarterly Sales'}


In [29]:
response

GenerateContentResponse(
  candidates=[
    Candidate(
      avg_logprobs=-0.5694650173187256,
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\xff\x02\x01\x8f=k_\x1e\xc0\x90\x8ePdKg\x1b7\xd3w\x1e"\xdb\x8aI\xf9B/$\xd7\xc6V\x7f`\xbb\x9d\xcf.\xef~\x88o5\x08\x93\'C\xda\x94\xb6\x13\xa3\x91^\xbb\xef\xfc\x86\xff\xa1\x9e4\xd3\xf9\xcc\x85k\x1a\xe2\xe4\x8cd\x08\xaaB\x0b\tn\x02\x1cK\x04\xeb\xf3\x18k\xb2\x13|l\xe7\xfc\x1c\x07*Ig...'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  create_time=datetime.datetime(2026, 1, 22, 3, 42, 18, 34381, tzinfo=TzInfo(0)),
  model_version='gemini-2.5-flash',
  response_id='mpxxac2MAq-Hz7sP3aj5uAU',
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    ca

## Function Execution ( The mannual way )

In [31]:
from google import genai
from google.genai import types


# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}





# Configure the client and tools
client = genai.Client()
tools = types.Tool(function_declarations=[set_light_values_declaration])
config = types.GenerateContentConfig(tools=[tools])

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Turn the lights down to a romantic level")]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)


print("================== Step 1: Model Suggests Function Call ==================")
print(response.candidates[0].content.parts[0].function_call)


# Extract tool call details, it may not be in the first part.
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "set_light_values":
    result = set_light_values(**tool_call.args)
    print("================== Step 2: Function Execution Result ==================")
    print(f"Function execution result: {result}")




# Create a function response part
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)
print("================== Step 4: Wrap Function Response ==================")
print(function_response_part)

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

print("================== Step 4a: Updated Contents ==================")
for content in contents:
    for part in content.parts:
        print(f'Role: {content.role}, Part: {part}')

client = genai.Client()
final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print("================== Step 5: Final Model Response ==================")
print(final_response.text)

================== Step 1: Model Suggests Function Call ==================
id=None args={'color_temp': 'warm', 'brightness': 25} name='set_light_values' partial_args=None will_continue=None
================== Step 2: Function Execution Result ==================
Function execution result: {'brightness': 25, 'colorTemperature': 'warm'}
================== Step 4: Wrap Function Response ==================
media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  name='set_light_values',
  response={
    'result': {
      'brightness': 25,
      'colorTemperature': 'warm'
    }
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None
================== Step 4a: Updated Contents ==================
Role: user, Part: media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='Turn the lights

## Parallel function calling

In addition to single turn function calling, you can also call multiple functions at once. Parallel function calling lets you execute multiple functions at once and is used when the functions are not dependent on each other. This is useful in scenarios like gathering data from multiple independent sources, such as retrieving customer details from different databases or checking inventory levels across various warehouses or performing multiple actions such as converting your apartment into a disco.

In [33]:
power_disco_ball = {
    "name": "power_disco_ball",
    "description": "Powers the spinning disco ball.",
    "parameters": {
        "type": "object",
        "properties": {
            "power": {
                "type": "boolean",
                "description": "Whether to turn the disco ball on or off.",
            }
        },
        "required": ["power"],
    },
}

start_music = {
    "name": "start_music",
    "description": "Play some music matching the specified parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "energetic": {
                "type": "boolean",
                "description": "Whether the music is energetic or not.",
            },
            "loud": {
                "type": "boolean",
                "description": "Whether the music is loud or not.",
            },
        },
        "required": ["energetic", "loud"],
    },
}

dim_lights = {
    "name": "dim_lights",
    "description": "Dim the lights.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "number",
                "description": "The brightness of the lights, 0.0 is off, 1.0 is full.",
            }
        },
        "required": ["brightness"],
    },
}

In [34]:
from google import genai
from google.genai import types

# Configure the client and tools
client = genai.Client()
house_tools = [
    types.Tool(function_declarations=[power_disco_ball, start_music, dim_lights])
]
config = types.GenerateContentConfig(
    tools=house_tools,
    automatic_function_calling=types.AutomaticFunctionCallingConfig(
        disable=True
    ),
    # Force the model to call 'any' function, instead of chatting.
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode='ANY')
    ),
)

chat = client.chats.create(model="gemini-2.5-flash", config=config)
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call
print("Example 1: Forced function calling")
for fn in response.function_calls:
    args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
    print(f"{fn.name}({args})")

Example 1: Forced function calling
power_disco_ball(power=True)
start_music(loud=True, energetic=True)
dim_lights(brightness=0.3)


In [35]:
response

GenerateContentResponse(
  candidates=[
    Candidate(
      avg_logprobs=-1.0244757652282714,
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\xba\x03\x01\x8f=k_\xd9\xde\xf6\x0c4>J\x88}e-\xd0g\n\xd1\xba\t\x8aX\xa2\t\xff\x17B\\WxO\xc3=O\x81\x11]/\xf6\xd3\x83\xe4\x7fZ\xe4C\x03Z\xcc\xa6\x9dH2\x81\xdaM\xcd\xd8\xcf\xfe\xb7\xbd\xefC3 \x95\t\x81\x92\x93(_VUS\xceuM\x1b\xfd\x1eK\xe5\xfa\x7f\xa7\xf3\\\xf3\xdf\x18...'
          ),
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            )
          ),
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            )
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.S

## Automatic Function Calls
The Python SDK supports automatic function calling, which automatically converts Python functions to declarations, handles the function call execution and response cycle for you. Following is an example for the disco use case.

In the mannual way we were doing

```
tools = types.Tool(function_declarations=[create_chart_function])
config = types.GenerateContentConfig(tools=[tools])
```

Now with automatic function calling we can do

```
config = types.GenerateContentConfig(
    tools=[power_disco_ball_impl, start_music_impl, dim_lights_impl]
)
```

In [43]:
from google import genai
from google.genai import types

power_disco_ball_state = None
start_music_state = None
dim_lights_state = None


# Actual function implementations
def power_disco_ball_impl(power: bool) -> dict:
    """Powers the spinning disco ball.

    Args:
        power: Whether to turn the disco ball on or off.

    Returns:
        A status dictionary indicating the current state.
    """
    print("Function called: power_disco_ball_impl")
    global power_disco_ball_state
    power_disco_ball_state =  {"status": f"Disco ball powered {'on' if power else 'off'}"}
    return {"status": f"Disco ball powered {'on' if power else 'off'}"}

def start_music_impl(energetic: bool, loud: bool) -> dict:
    """Play some music matching the specified parameters.

    Args:
        energetic: Whether the music is energetic or not.
        loud: Whether the music is loud or not.

    Returns:
        A dictionary containing the music settings.
    """
    print("Function called: start_music_impl")
    music_type = "energetic" if energetic else "chill"
    volume = "loud" if loud else "quiet"
    global start_music_state
    start_music_state = {"music_type": music_type, "volume": volume}
    return {"music_type": music_type, "volume": volume}

def dim_lights_impl(brightness: float) -> dict:
    """Dim the lights.

    Args:
        brightness: The brightness of the lights, 0.0 is off, 1.0 is full.

    Returns:
        A dictionary containing the new brightness setting.
    """
    print("Function called: dim_lights_impl")
    global dim_lights_state
    dim_lights_state = {"brightness": brightness}
    return {"brightness": brightness}

# Configure the client
client = genai.Client()
config = types.GenerateContentConfig(
    tools=[power_disco_ball_impl, start_music_impl, dim_lights_impl]
)

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party!",
    config=config,
)

print("\nAutomatic function calling")
print(response.text)

Function called: power_disco_ball_impl
Function called: start_music_impl
Function called: dim_lights_impl

Automatic function calling
Alright, the party is starting! The disco ball is spinning, energetic and loud music is playing, and the lights have been dimmed to 30%. Let's get this party going!


In [41]:
power_disco_ball_state, start_music_state, dim_lights_state

({'status': 'Disco ball powered on'},
 {'music_type': 'energetic', 'volume': 'loud'},
 {'brightness': 0.3})

#### Trying to manipulate the order of Execution

In [44]:
# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party! First start the music loudly and energetically, then turn on the disco ball, and finally dim the lights to 50% brightness.",
    config=config,
)

print("\nAutomatic function calling")
print(response.text)

Function called: start_music_impl
Function called: power_disco_ball_impl
Function called: dim_lights_impl

Automatic function calling
Party started! The music is loud and energetic, the disco ball is spinning, and the lights are at 50% brightness.


In [45]:
# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party! First start the music loudly and energetically, then turn on the disco ball, and finally dim the lights to 50% brightness. Then again start the music quietly and chill, turn off the disco ball, and set the lights to full brightness.",
    config=config,
)

print("\nAutomatic function calling")
print(response.text)

Function called: start_music_impl
Function called: power_disco_ball_impl
Function called: dim_lights_impl
Function called: start_music_impl
Function called: power_disco_ball_impl
Function called: dim_lights_impl

Automatic function calling
Alright, I've set up the party: energetic and loud music, disco ball on, and lights at 50%.

Then, I've wound things down: chill and quiet music, disco ball off, and lights at full brightness.


In [46]:
power_disco_ball_state, start_music_state, dim_lights_state

({'status': 'Disco ball powered off'},
 {'music_type': 'chill', 'volume': 'quiet'},
 {'brightness': 1})

## Compositional function calling

Passing the output of one function as the input to another function. This is useful when the functions are dependent on each other and the output of one function is needed as input for another function. For example, you can first retrieve user information and then use that information to generate a personalized recommendation.

In [47]:
import os
from google import genai
from google.genai import types

# Example Functions
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_weather_forecast(location={location})")
    # TODO: Make API call
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 25, "unit": "celsius"}  # Dummy response

def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    # TODO: Interact with a thermostat API
    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

# Configure the client and model
client = genai.Client()
config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature]
)

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    config=config,
)

# Print the final, user-facing response
print(response.text)

Tool Call: get_weather_forecast(location=London)
Tool Response: {'temperature': 25, 'unit': 'celsius'}
Tool Call: set_thermostat_temperature(temperature=20)
Tool Response: {'status': 'success'}
The thermostat has been set to 20°C.


## Final Notes for Function Calling

### Modes
The Gemini API lets you control how the model uses the provided tools (function declarations). Specifically, you can set the mode within the.function_calling_config.

-AUTO (Default): The model decides whether to generate a natural language response or suggest a function call based on the prompt and context. This is the most flexible mode and recommended for most scenarios.
- ANY: The model is constrained to always predict a function call and guarantees function schema adherence. If allowed_function_names is not specified, the model can choose from any of the provided function declarations. If allowed_function_names is provided as a list, the model can only choose from the functions in that list. Use this mode when you require a function call response to every prompt (if applicable).
- NONE: The model is prohibited from making function calls. This is equivalent to sending a request without any function declarations. Use this to temporarily disable function calling without removing your tool definitions.
- VALIDATED (Preview): The model is constrained to predict either function calls or natural language, and ensures function schema adherence. If allowed_function_names is not provided, the model picks from all of the available function declarations. If allowed_function_names is provided, the model picks from the set of allowed functions.

Example usage:

```
# Configure function calling mode
tool_config = types.ToolConfig(
    function_calling_config=types.FunctionCallingConfig(
        mode="ANY", allowed_function_names=["get_current_temperature"]
    )
)

# Create the generation config
config = types.GenerateContentConfig(
    tools=[tools],  # not defined here.
    tool_config=tool_config,
)
```


### Combine native tools with function calling

Multi-tool use is a-Live API only feature at the moment. The run() function declaration, which handles the asynchronous websocket setup, is omitted for brevity.

```
# Multiple tasks example - combining lights, code execution, and search
prompt = """
  Hey, I need you to do three things for me.

    1.  Turn on the lights.
    2.  Then compute the largest prime palindrome under 100000.
    3.  Then use Google Search to look up information about the largest earthquake in California the week of Dec 5 2024.

  Thanks!
  """

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights_schema, turn_off_the_lights_schema]} # not defined here.
]

# Execute the prompt with specified tools in audio modality
await run(prompt, tools=tools, modality="AUDIO")
```

### Live API: https://ai.google.dev/gemini-api/docs/live?example=mic-stream


## Code Execution

The Gemini API provides a code execution tool that enables the model to generate and run Python code. The model can then learn iteratively from the code execution results until it arrives at a final output. You can use code execution to build applications that benefit from code-based reasoning. For example, you can use code execution to solve equations or process text. You can also use the libraries included in the code execution environment to perform more specialized tasks.

Gemini is only able to execute code in Python. You can still ask Gemini to generate code in another language, but the model can't use the code execution tool to run it.

https://ai.google.dev/gemini-api/docs/code-execution

- Supported libraries
The code execution environment includes the following libraries:

attrs
chess
contourpy
fpdf
geopandas
imageio
jinja2
joblib
jsonschema
jsonschema-specifications
lxml
matplotlib
mpmath
numpy
opencv-python
openpyxl
packaging
pandas
pillow
protobuf
pylatex
pyparsing
PyPDF2
python-dateutil
python-docx
python-pptx
reportlab
scikit-learn
scipy
seaborn
six
striprtf
sympy
tabulate
tensorflow
toolz
xlrd

###### You can't install your own libraries.



In [48]:
from google import genai
from google.genai import types

client = genai.Client()

chat = client.chats.create(
    model="gemini-3-flash-preview",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

response = chat.send_message("I have a math question for you.")
print(response.text)

response = chat.send_message(
    "What is the sum of the first 50 prime numbers? "
    "Generate and run code for the calculation, and make sure you get all 50."
)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    if part.executable_code is not None:
        print(part.executable_code.code)
    if part.code_execution_result is not None:
        print(part.code_execution_result.output)

I'm ready! Go ahead and share it with me. Whether it's algebra, calculus, geometry, statistics, or anything else, I'll do my best to help you solve it or explain the concepts involved.
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = []
num = 2
while len(primes) < 50:
    if is_prime(num):
        primes.append(num)
    num += 1

print(f"The first 50 primes are: {primes}")
print(f"The number of primes found: {len(primes)}")
print(f"The sum of the first 50 primes is: {sum(primes)}")
The first 50 primes are: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
The number of primes found: 50
The sum of the first 50 primes is: 5117

The sum of the first 50 prime numbers is **5,117**.

To find this, I used a s